In [27]:
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import TimeSeriesSplit
from scipy import stats
from collections import Counter
from sklearn.metrics import accuracy_score

In [28]:
weather = pd.read_csv("../jena.csv")

In [29]:
weather

,Date Time,p (mbar),T (degC),Tpot (K),Tdew (degC),rh (%),VPmax (mbar),VPact (mbar),VPdef (mbar),sh (g/kg),H2OC (mmol/mol),rho (g/m**3),wv (m/s),max. wv (m/s),wd (deg)
0,01.01.2009 00:10:00,996.52,-8.02,265.40,-8.90,93.30,3.33,3.11,0.22,1.94,3.12,1307.75,1.03,1.75,152.3
1,01.01.2009 00:20:00,996.57,-8.41,265.01,-9.28,93.40,3.23,3.02,0.21,1.89,3.03,1309.80,0.72,1.50,136.1
2,01.01.2009 00:30:00,996.53,-8.51,264.91,-9.31,93.90,3.21,3.01,0.20,1.88,3.02,1310.24,0.19,0.63,171.6
3,01.01.2009 00:40:00,996.51,-8.31,265.12,-9.07,94.20,3.26,3.07,0.19,1.92,3.08,1309.19,0.34,0.50,198.0
4,01.01.2009 00:50:00,996.51,-8.27,265.15,-9.04,94.10,3.27,3.08,0.19,1.92,3.09,1309.00,0.32,0.63,214.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420546,31.12.2016 23:20:00,1000.07,-4.05,269.10,-8.13,73.10,4.52,3.30,1.22,2.06,3.30,1292.98,0.67,1.52,240.0
420547,31.12.2016 23:30:00,999.93,-3.35,269.81,-8.06,69.71,4.77,3.32,1.44,2.07,3.32,1289.44,1.14,1.92,234.3
420548,31.12.2016 23:40:00,999.82,-3.16,270.01,-8.21,67.91,4.84,3.28,1.55,2.05,3.28,1288.39,1.08,2.00,215.2
420549,31.12.2016 23:50:00,999.81,-4.23,268.94,-8.53,71.80,4.46,3.20,1.26,1.99,3.20,1293.56,1.49,2.16,225.8


In [30]:
weather['tempmax_next'] = weather['T (degC)'].shift(-1)

In [31]:
weather

,Date Time,p (mbar),T (degC),Tpot (K),Tdew (degC),rh (%),VPmax (mbar),VPact (mbar),VPdef (mbar),sh (g/kg),H2OC (mmol/mol),rho (g/m**3),wv (m/s),max. wv (m/s),wd (deg),tempmax_next
0,01.01.2009 00:10:00,996.52,-8.02,265.40,-8.90,93.30,3.33,3.11,0.22,1.94,3.12,1307.75,1.03,1.75,152.3,-8.41
1,01.01.2009 00:20:00,996.57,-8.41,265.01,-9.28,93.40,3.23,3.02,0.21,1.89,3.03,1309.80,0.72,1.50,136.1,-8.51
2,01.01.2009 00:30:00,996.53,-8.51,264.91,-9.31,93.90,3.21,3.01,0.20,1.88,3.02,1310.24,0.19,0.63,171.6,-8.31
3,01.01.2009 00:40:00,996.51,-8.31,265.12,-9.07,94.20,3.26,3.07,0.19,1.92,3.08,1309.19,0.34,0.50,198.0,-8.27
4,01.01.2009 00:50:00,996.51,-8.27,265.15,-9.04,94.10,3.27,3.08,0.19,1.92,3.09,1309.00,0.32,0.63,214.3,-8.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420546,31.12.2016 23:20:00,1000.07,-4.05,269.10,-8.13,73.10,4.52,3.30,1.22,2.06,3.30,1292.98,0.67,1.52,240.0,-3.35
420547,31.12.2016 23:30:00,999.93,-3.35,269.81,-8.06,69.71,4.77,3.32,1.44,2.07,3.32,1289.44,1.14,1.92,234.3,-3.16
420548,31.12.2016 23:40:00,999.82,-3.16,270.01,-8.21,67.91,4.84,3.28,1.55,2.05,3.28,1288.39,1.08,2.00,215.2,-4.23
420549,31.12.2016 23:50:00,999.81,-4.23,268.94,-8.53,71.80,4.46,3.20,1.26,1.99,3.20,1293.56,1.49,2.16,225.8,-4.82


In [32]:
weather = weather[:-1]

In [33]:
weather

,Date Time,p (mbar),T (degC),Tpot (K),Tdew (degC),rh (%),VPmax (mbar),VPact (mbar),VPdef (mbar),sh (g/kg),H2OC (mmol/mol),rho (g/m**3),wv (m/s),max. wv (m/s),wd (deg),tempmax_next
0,01.01.2009 00:10:00,996.52,-8.02,265.40,-8.90,93.30,3.33,3.11,0.22,1.94,3.12,1307.75,1.03,1.75,152.3,-8.41
1,01.01.2009 00:20:00,996.57,-8.41,265.01,-9.28,93.40,3.23,3.02,0.21,1.89,3.03,1309.80,0.72,1.50,136.1,-8.51
2,01.01.2009 00:30:00,996.53,-8.51,264.91,-9.31,93.90,3.21,3.01,0.20,1.88,3.02,1310.24,0.19,0.63,171.6,-8.31
3,01.01.2009 00:40:00,996.51,-8.31,265.12,-9.07,94.20,3.26,3.07,0.19,1.92,3.08,1309.19,0.34,0.50,198.0,-8.27
4,01.01.2009 00:50:00,996.51,-8.27,265.15,-9.04,94.10,3.27,3.08,0.19,1.92,3.09,1309.00,0.32,0.63,214.3,-8.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420545,31.12.2016 23:10:00,1000.11,-3.93,269.23,-8.09,72.60,4.56,3.31,1.25,2.06,3.31,1292.41,0.56,1.00,202.6,-4.05
420546,31.12.2016 23:20:00,1000.07,-4.05,269.10,-8.13,73.10,4.52,3.30,1.22,2.06,3.30,1292.98,0.67,1.52,240.0,-3.35
420547,31.12.2016 23:30:00,999.93,-3.35,269.81,-8.06,69.71,4.77,3.32,1.44,2.07,3.32,1289.44,1.14,1.92,234.3,-3.16
420548,31.12.2016 23:40:00,999.82,-3.16,270.01,-8.21,67.91,4.84,3.28,1.55,2.05,3.28,1288.39,1.08,2.00,215.2,-4.23


In [34]:
features = ['p (mbar)',	'T (degC)',	'Tpot (K)',	'Tdew (degC)',	'rh (%)'	,'VPmax (mbar)',	'VPact (mbar)',	'VPdef (mbar)',	'sh (g/kg)',	'H2OC (mmol/mol)',	'rho (g/m**3)',	'wv (m/s)',	'max. wv (m/s)',	'wd (deg)']
X = weather[features]
y = weather['tempmax_next']

In [35]:
weather

,Date Time,p (mbar),T (degC),Tpot (K),Tdew (degC),rh (%),VPmax (mbar),VPact (mbar),VPdef (mbar),sh (g/kg),H2OC (mmol/mol),rho (g/m**3),wv (m/s),max. wv (m/s),wd (deg),tempmax_next
0,01.01.2009 00:10:00,996.52,-8.02,265.40,-8.90,93.30,3.33,3.11,0.22,1.94,3.12,1307.75,1.03,1.75,152.3,-8.41
1,01.01.2009 00:20:00,996.57,-8.41,265.01,-9.28,93.40,3.23,3.02,0.21,1.89,3.03,1309.80,0.72,1.50,136.1,-8.51
2,01.01.2009 00:30:00,996.53,-8.51,264.91,-9.31,93.90,3.21,3.01,0.20,1.88,3.02,1310.24,0.19,0.63,171.6,-8.31
3,01.01.2009 00:40:00,996.51,-8.31,265.12,-9.07,94.20,3.26,3.07,0.19,1.92,3.08,1309.19,0.34,0.50,198.0,-8.27
4,01.01.2009 00:50:00,996.51,-8.27,265.15,-9.04,94.10,3.27,3.08,0.19,1.92,3.09,1309.00,0.32,0.63,214.3,-8.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420545,31.12.2016 23:10:00,1000.11,-3.93,269.23,-8.09,72.60,4.56,3.31,1.25,2.06,3.31,1292.41,0.56,1.00,202.6,-4.05
420546,31.12.2016 23:20:00,1000.07,-4.05,269.10,-8.13,73.10,4.52,3.30,1.22,2.06,3.30,1292.98,0.67,1.52,240.0,-3.35
420547,31.12.2016 23:30:00,999.93,-3.35,269.81,-8.06,69.71,4.77,3.32,1.44,2.07,3.32,1289.44,1.14,1.92,234.3,-3.16
420548,31.12.2016 23:40:00,999.82,-3.16,270.01,-8.21,67.91,4.84,3.28,1.55,2.05,3.28,1288.39,1.08,2.00,215.2,-4.23


In [36]:
features

['p (mbar)',
 'T (degC)',
 'Tpot (K)',
 'Tdew (degC)',
 'rh (%)',
 'VPmax (mbar)',
 'VPact (mbar)',
 'VPdef (mbar)',
 'sh (g/kg)',
 'H2OC (mmol/mol)',
 'rho (g/m**3)',
 'wv (m/s)',
 'max. wv (m/s)',
 'wd (deg)']

In [37]:
def entropy(s):
    values, counts = np.unique(s, return_counts=True)
    probabilities = counts / counts.sum()
    entropy = -np.sum(probabilities * np.log2(probabilities))
    return entropy

In [38]:
X = weather[features]
y = weather['tempmax_next']

In [39]:
print("Số lượng giá trị khuyết trong X:")
print(X.isnull().sum())
print("\nSố lượng giá trị khuyết trong y:")
print(y.isnull().sum())

Số lượng giá trị khuyết trong X:
p (mbar)           0
T (degC)           0
Tpot (K)           0
Tdew (degC)        0
rh (%)             0
VPmax (mbar)       0
VPact (mbar)       0
VPdef (mbar)       0
sh (g/kg)          0
H2OC (mmol/mol)    0
rho (g/m**3)       0
wv (m/s)           0
max. wv (m/s)      0
wd (deg)           0
dtype: int64

Số lượng giá trị khuyết trong y:
0


In [40]:
weather_clean = weather.dropna(subset=features + ['tempmax_next'])
X = weather_clean[features]
y = weather_clean['tempmax_next']

In [41]:
print("Số lượng giá trị khuyết trong X:")
print(X.isnull().sum())
print("\nSố lượng giá trị khuyết trong y:")
print(y.isnull().sum())

Số lượng giá trị khuyết trong X:
p (mbar)           0
T (degC)           0
Tpot (K)           0
Tdew (degC)        0
rh (%)             0
VPmax (mbar)       0
VPact (mbar)       0
VPdef (mbar)       0
sh (g/kg)          0
H2OC (mmol/mol)    0
rho (g/m**3)       0
wv (m/s)           0
max. wv (m/s)      0
wd (deg)           0
dtype: int64

Số lượng giá trị khuyết trong y:
0


In [42]:
# Kiểm tra kiểu dữ liệu của các cột trong `X`
print("\nKiểu dữ liệu của các cột trong X:")
print(X.dtypes)



Kiểu dữ liệu của các cột trong X:
p (mbar)           float64
T (degC)           float64
Tpot (K)           float64
Tdew (degC)        float64
rh (%)             float64
VPmax (mbar)       float64
VPact (mbar)       float64
VPdef (mbar)       float64
sh (g/kg)          float64
H2OC (mmol/mol)    float64
rho (g/m**3)       float64
wv (m/s)           float64
max. wv (m/s)      float64
wd (deg)           float64
dtype: object


In [43]:
# Thống kê mô tả cho `X`
print("\nThống kê mô tả cho X:")
print(X.describe())

# Thống kê mô tả cho `y`
print("\nThống kê mô tả cho y:")
print(y.describe())


Thống kê mô tả cho X:
            p (mbar)       T (degC)       Tpot (K)    Tdew (degC)  \
count  420550.000000  420550.000000  420550.000000  420550.000000   
mean      989.212751       9.450181     283.492779       4.955886   
std         8.358475       8.423346       8.504449       6.730651   
min       913.600000     -23.010000     250.600000     -25.010000   
25%       984.200000       3.360000     277.430000       0.240000   
50%       989.580000       9.420000     283.470000       5.220000   
75%       994.720000      15.470000     289.530000      10.070000   
max      1015.350000      37.280000     311.340000      23.110000   

              rh (%)   VPmax (mbar)   VPact (mbar)   VPdef (mbar)  \
count  420550.000000  420550.000000  420550.000000  420550.000000   
mean       76.008260      13.576273       9.533771       4.042419   
std        16.476195       7.739016       4.184158       4.896855   
min        12.950000       0.950000       0.790000       0.000000   
25%       

In [44]:
from sklearn.model_selection import train_test_split



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Kích thước tập huấn luyện:", X_train.shape, y_train.shape)
print("Kích thước tập kiểm thử:", X_test.shape, y_test.shape)

Kích thước tập huấn luyện: (336440, 14) (336440,)
Kích thước tập kiểm thử: (84110, 14) (84110,)


In [45]:
print("Số lượng giá trị khuyết trong y sau khi loại bỏ:")
print(y.isnull().sum())

Số lượng giá trị khuyết trong y sau khi loại bỏ:
0


In [46]:
import numpy as np

class DecisionTreeRegressor:
    def __init__(self, max_depth=None, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.left = None
        self.right = None
        self.threshold = None
        self.feature_index = None
        self.value = None

    def fit(self, X, y, depth=0):
        num_samples, num_features = X.shape
        if num_samples >= self.min_samples_split and (self.max_depth is None or depth < self.max_depth):
            best_split = self._get_best_split(X, y, num_samples, num_features)
            if best_split:
                self.feature_index, self.threshold = best_split
                indices_left = X[:, self.feature_index] < self.threshold
                X_left, y_left = X[indices_left], y[indices_left]
                X_right, y_right = X[~indices_left], y[~indices_left]
                self.left = DecisionTreeRegressor(self.max_depth, self.min_samples_split)
                self.right = DecisionTreeRegressor(self.max_depth, self.min_samples_split)
                self.left.fit(X_left, y_left, depth + 1)
                self.right.fit(X_right, y_right, depth + 1)
        self.value = np.mean(y)

    def _get_best_split(self, X, y, num_samples, num_features):
        best_mse = float("inf")
        best_index = None
        best_threshold = None
        
        for feature_index in range(num_features):
            feature_values = X[:, feature_index]
            sorted_indices = np.argsort(feature_values)
            feature_values, labels = feature_values[sorted_indices], y[sorted_indices]
            
            for i in range(1, num_samples):
                if feature_values[i] == feature_values[i - 1]:
                    continue
                split_threshold = (feature_values[i] + feature_values[i - 1]) / 2
                left_labels, right_labels = labels[:i], labels[i:]
                mse = self._calculate_mse(left_labels, right_labels)
                
                if mse < best_mse:
                    best_mse = mse
                    best_index = feature_index
                    best_threshold = split_threshold
        
        return (best_index, best_threshold) if best_index is not None else None

    def _calculate_mse(self, left_labels, right_labels):
        left_var = np.var(left_labels) * len(left_labels)
        right_var = np.var(right_labels) * len(right_labels)
        return left_var + right_var

    def predict(self, X):
        if self.left is None or self.right is None:
            return np.array([self.value] * len(X))
        
        indices = X[:, self.feature_index] < self.threshold
        left_preds = self.left.predict(X[indices])
        right_preds = self.right.predict(X[~indices])
        
        return np.concatenate([left_preds, right_preds])



In [47]:
class RandomForestRegressor:
    def __init__(self, n_estimators=100, max_depth=None, min_samples_split=2, random_state=None):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.random_state = random_state
        self.trees = []
        if random_state is not None:
            np.random.seed(random_state)

    def fit(self, X, y):
        self.trees = []
        for _ in range(self.n_estimators):
            bootstrap_indices = np.random.choice(len(X), size=len(X), replace=True)
            X_sample, y_sample = X[bootstrap_indices], y[bootstrap_indices]
            tree = DecisionTreeRegressor(self.max_depth, self.min_samples_split)
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)

    def predict(self, X):
        tree_predictions = np.array([tree.predict(X) for tree in self.trees])
        return np.mean(tree_predictions, axis=0)

In [49]:
from sklearn.ensemble import RandomForestRegressor

# Khởi tạo mô hình Random Forest
model = RandomForestRegressor(n_estimators=1000, random_state=42)  # Sử dụng 100 cây

# Huấn luyện mô hình trên tập huấn luyện
model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [50]:
from sklearn.metrics import mean_squared_error, r2_score

# Dự đoán nhiệt độ trên tập kiểm thử
y_pred = model.predict(X_test)

# Đánh giá mô hình bằng Mean Squared Error và R-squared
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)

Mean Squared Error: 0.06153145782235228
R-squared: 0.9991360368403509


In [51]:
weather

,Date Time,p (mbar),T (degC),Tpot (K),Tdew (degC),rh (%),VPmax (mbar),VPact (mbar),VPdef (mbar),sh (g/kg),H2OC (mmol/mol),rho (g/m**3),wv (m/s),max. wv (m/s),wd (deg),tempmax_next
0,01.01.2009 00:10:00,996.52,-8.02,265.40,-8.90,93.30,3.33,3.11,0.22,1.94,3.12,1307.75,1.03,1.75,152.3,-8.41
1,01.01.2009 00:20:00,996.57,-8.41,265.01,-9.28,93.40,3.23,3.02,0.21,1.89,3.03,1309.80,0.72,1.50,136.1,-8.51
2,01.01.2009 00:30:00,996.53,-8.51,264.91,-9.31,93.90,3.21,3.01,0.20,1.88,3.02,1310.24,0.19,0.63,171.6,-8.31
3,01.01.2009 00:40:00,996.51,-8.31,265.12,-9.07,94.20,3.26,3.07,0.19,1.92,3.08,1309.19,0.34,0.50,198.0,-8.27
4,01.01.2009 00:50:00,996.51,-8.27,265.15,-9.04,94.10,3.27,3.08,0.19,1.92,3.09,1309.00,0.32,0.63,214.3,-8.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420545,31.12.2016 23:10:00,1000.11,-3.93,269.23,-8.09,72.60,4.56,3.31,1.25,2.06,3.31,1292.41,0.56,1.00,202.6,-4.05
420546,31.12.2016 23:20:00,1000.07,-4.05,269.10,-8.13,73.10,4.52,3.30,1.22,2.06,3.30,1292.98,0.67,1.52,240.0,-3.35
420547,31.12.2016 23:30:00,999.93,-3.35,269.81,-8.06,69.71,4.77,3.32,1.44,2.07,3.32,1289.44,1.14,1.92,234.3,-3.16
420548,31.12.2016 23:40:00,999.82,-3.16,270.01,-8.21,67.91,4.84,3.28,1.55,2.05,3.28,1288.39,1.08,2.00,215.2,-4.23


In [55]:
data_30_4 = {
    'p (mbar)': [999.82],	
    'T (degC)': [-4.82],	
    'Tpot (K)': [268.36],	
    'Tdew (degC)': [-8.42],	
    'rh (%)': [75.70]	,
    'VPmax (mbar)': [4.27],	
    'VPact (mbar)': [3.23],	
    'VPdef (mbar)': [1.04],	
    'sh (g/kg)': [2.01],	
    'H2OC (mmol/mol)': [3.23],	
    'rho (g/m**3)': [1296.38],	
    'wv (m/s)': [1.23],	
    'max. wv (m/s)': [1.96],	
    'wd (deg)': [184.9],
}

new_data = pd.DataFrame(data_30_4)

In [56]:
predicted_tempmax = model.predict(new_data)
print(f"Dự đoán nhiệt độ cao nhất cho ngày 1 tháng 5 tại Hà Nội là: {predicted_tempmax[0]}°C")

Dự đoán nhiệt độ cao nhất cho ngày 1 tháng 5 tại Hà Nội là: -4.860199999999999°C
